Hier worden alle libraries geïmporteerd

In [ ]:
# importing required libraries

import pandas as pd
from pandas import read_csv
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import datasets
from sklearn import tree
import warnings
warnings.simplefilter("ignore")


Hier wordt alle data van de dataset opgehaald

In [ ]:
# importing the dataset

result_df = pd.read_csv("./data/results.csv")
stats_df = pd.read_csv("./data/status.csv")
drivers_df = pd.read_csv("./data/drivers.csv")
races_df = pd.read_csv("./data/races.csv")
constructor_df = pd.read_csv("./data/constructors.csv")
driver_standings_df = pd.read_csv("./data/driver_standings.csv")
qualifying_df = pd.read_csv("./data/qualifying.csv")


De data wordt weergegeven om te checken of het ophalen hiervan goed is gegaan

In [ ]:
# Viewing the data
result_df.head()


Alle data wordt samengevoegd tot 1 dataframe en daarna wordt deze weergegeven om te checken of alles goed is gegaan

In [ ]:
# merging all seperate dataframe into single dataframe as df
con1 = pd.merge(result_df, races_df, on='raceId')
con2 = pd.merge(con1, drivers_df, on='driverId')
con3 = pd.merge(con2, driver_standings_df, on=['driverId', 'raceId'])
con4 = pd.merge(con3, constructor_df, on='constructorId')
con5 = pd.merge(con4, qualifying_df, on=['raceId', 'driverId'])
df = pd.merge(con5, stats_df, on=['statusId'])
pd.get_option("display.max_columns", None)

# checking the data types
df.head()


Er wordt gekeken welke data types er in de dataframe zitten, en hoeveel RAM ze gebruiken

In [ ]:
# checking what types of data are in the dataframe and how much ram they take up
df.info()


Alle kollomen die in het dataframe staan worden hier geprint

In [ ]:
# viewing the columns in the dataframe
df.columns


Alle kollomen die geen waardevolle data hebben worden hier gedropt

In [ ]:
# dropping unnecessary columns
df = df.drop(['url', 'url_x', 'fastestLapTime', 'positionText_x', 'time_x', 'time_y', 'driverRef', 'constructorRef', 'nationality_y', 'url_y', 'positionText_y', 'points_y', 'rank', 'number_y', 'milliseconds', 'fastestLapSpeed',
             'number_x', 'code', 'fastestLap', 'driverStandingsId', 'q1', 'q2', 'q3', 'status', 'constructorId_x', 'constructorId_y', 'points_x', 'qualifyId', 'wins', 'resultId', 'positionOrder', 'position_y', 'grid', 'statusId'], axis='columns')


Alle kollomen met onduidelijke namen worden hernoemd naar namen die duidelijk aangeven welke data er in staat

In [ ]:
# renaming the column names to less confusing names

col_name = {'name_x': 'grand_prix', 'nationality_x': 'nationality', 'name_y': 'constructor',
            'raceId_x': 'racerId', 'points_x': 'points', 'forename': 'firstname', 'position_x': 'finnishPosition'}

df.rename(columns=col_name, inplace=True)
df.head()


Twee kolommen die los van elkaar overbodige kollomen vormen worden samen gevoegd tot 1 zinvolle en bruikbare kolom

In [ ]:
# combining the two columns into one column for better readability

df['driver_name'] = df.pop('firstname')+' '+df.pop('surname')


Er wordt gekeken of het samenvoegen gelukt is

In [ ]:
df.head()


De kollomen "dob" en "date" worden van een string omgezet naar een datetime format

In [ ]:
# turning string colums into datetime columns

df['dob'] = pd.to_datetime(df['dob'])
df['date'] = pd.to_datetime(df['date'])


Date en dob worden hier samengevoegd tot de kollom "age", "age" wordt berekend door de verkregen waardes te delen door 365 en daarna af te ronden om de data relationeler te maken. Hierna worden de eerste kollomen geprint om te checken of het gelukt is

In [ ]:
# calculating driver's age and creating it as a new column

dates = df.pop('date')-df.pop('dob')
age = dates.dt.days/365


# rounding the age to the nearest year for better relational analysis

df['age'] = round(age)
pd.set_option('display.max_columns', None)

df.head()


Er wordt aangetoond data het verwerken van de data zorgt voor minder RAM gebruik

In [ ]:
# showing that the data processing lessens the ram usage (from 49 columns to 13 columns)

df.info()


Er wordt gekeken of er null waardes in het dataframe staan, sinds dat niet het geval is hoeft er niet met null waardes te worden gewerkt

In [ ]:
# checking for null values

df.isnull().sum() / len(df) * 100


# since unused there are no null values in this dataframe there has to be no handling of null values


De functie labelencoder zet catagorische variabelen om naar nummerieke (coderen) zodat de computer ze kan interpeteren

In [ ]:
# doing necessary imports to let the machine actually understand the data

le = LabelEncoder()


Waardes onder 2010 worden niet meegenomen omdat deze niet representatief zijn voor nieuwere berekeningen

In [ ]:
# dropping years before 2010 because there was a new points system introduced so previous data is not accurate anymore

df.drop(df[df.year < 2010].index, inplace=True)

Kolommen worden ingedeeld in categorische kolommen en nummerieke kolommen

In [ ]:
# seperating categorical and numerical columns for understading

cat = []
num = []
for i in df.columns:
    if df[i].dtypes == 'O':
        cat.append(i)
    else:
        num.append(i)


Categorische tabellen worden omgezet naar nummerieke waarden door middel van codering

In [ ]:
# encoding categorical columns

for i in cat:
    df[i] = le.fit_transform(df[i])
df.head()


De index van de database wordt opnieuw ingedeeld zodat deze met het nieuwe dataframe beter leesbaar is

In [ ]:
df = df.reset_index()

Het preprocessen van de data is afgelopen en het dataframe wordt in een .csv opgeslagen

In [ ]:
df.to_csv(r'./newdata/collectionfile.csv')

Het definieren van de waarden van de decisiontree aan een variabele (model)

In [ ]:
# defining the tree structure

model = tree.DecisionTreeRegressor(max_depth=5, random_state=1234)


Het definieren van de waarden die gebruikt worden om te voorspellen en de te voorspellen waarde

In [ ]:
# creating two seperate dataframes for the training and testing data

X = pd.DataFrame(df.drop(['finnishPosition'], axis=1))

y = pd.DataFrame(df, columns=['finnishPosition'])


De data splitten in trainingdata en testdata met de gegeven parameters

In [ ]:
# performing a train test split to separate the data into training and testing data

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y, test_size=0.33, random_state=42)


Past de classificationTree aan, aan de invoertrainingsgegevens

In [ ]:
model.fit(Xtrain, ytrain)

Scoort het model van de kolom boven op overeenkomst

In [ ]:
model.score(Xtrain, ytrain)


Past de classificationTree aan, aan de invoertrainingsgegevens

In [ ]:
model.fit(Xtest, ytest)

Scoort het model van de kolom boven op overeenkomst

In [ ]:
model.score(Xtest, ytest)


Voorspelling maken van y op basis van Xtest

In [ ]:
y_pred_tree = model.predict(Xtest)


Het vergelijken van de ytest- en de voorspelde y waarden

In [ ]:
r2_score(ytest, y_pred_tree)

Een grafiek maken die de werkelijke data plot tegen de voorspelde data

In [ ]:
# visualising the first 50 rows of the data vs the predicted values

figGraph = plt.figure(figsize=(10, 10))

plt.plot([item for item in range(len(ytest[0:50]))], ytest.values[0:50], label="Actual Data", linestyle=':')
plt.plot([item for item in range(len(ytest[0:50]))], y_pred_tree[0:50], label="Predicted Data")
plt.ylabel("FinnishPosition")
plt.xlabel("Drivers")
plt.legend(loc='best')
plt.show()


Het opslaan van de grafiek

In [ ]:
figGraph.savefig("./Figures/predictionVSreality_graph.png")

De variable feature_names definieren als X om het te plotten in een decisiontree

In [ ]:
# defining the variables for the tree

fn = list(X)


Het plotten van de tree

In [ ]:
# creating the tree

treeFig = plt.figure(figsize=(150,150))
_ = tree.plot_tree(model,
               feature_names=fn,
               filled=True)

Het opslaan van de geplotte tree

In [ ]:
# saving the plotted tree as a png file

treeFig.savefig("./Figures/decision_tree.png")